# Rule Testing

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from itertools import groupby
from collections import Counter
# from jupyterthemes import jtplot
# jtplot.reset()
import os
import glob
from copy import deepcopy

In [3]:
""" Reads data in from all participants. The final virtual game data from 
In the Zone and Simon Says. Only takes data where the entry type = 0. 
python lists: 'InTheZone' and 'SimonSays'
"""

path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "../Data/All_Participants/InTheZone/*"))
InTheZone = []
print("In the Zone Paths:")

for i in csv_files:
    df = pd.read_csv(i)
    df = df[df['entryType'] == 0]
    InTheZone.append(df)
    print(i)
    
csv_files = glob.glob(os.path.join(path, "../Data/All_Participants/SimonSays/*"))
SimonSays = []
print("Simon Says Paths:")

for i in csv_files:
    df = pd.read_csv(i)
    df = df[df['entryType'] == 0]
    SimonSays.append(df)
    print(i)

In the Zone Paths:
/home/androo/Documents/Coapt/Code/../Data/All_Participants/InTheZone/32098021_VirtualArmGames_-2_18_2021_10_3_48.csv
/home/androo/Documents/Coapt/Code/../Data/All_Participants/InTheZone/52054922_VirtualArmGames_-3_2_2021_8_52_40.csv
/home/androo/Documents/Coapt/Code/../Data/All_Participants/InTheZone/51013322_VirtualArmGames_-5_29_2021_17_24_48.csv
/home/androo/Documents/Coapt/Code/../Data/All_Participants/InTheZone/32136722_VirtualArmGames_-5_18_2021_20_18_32.csv
/home/androo/Documents/Coapt/Code/../Data/All_Participants/InTheZone/32068222_VirtualArmGames_-5_11_2021_19_57_6.csv
/home/androo/Documents/Coapt/Code/../Data/All_Participants/InTheZone/32132721_VirtualArmGames_-11_24_2021_10_53_46.csv
/home/androo/Documents/Coapt/Code/../Data/All_Participants/InTheZone/32195432_VirtualArmGames_-3_19_2021_14_40_11.csv
/home/androo/Documents/Coapt/Code/../Data/All_Participants/InTheZone/51048532_VirtualArmGames_-2_9_2021_11_53_0.csv
/home/androo/Documents/Coapt/Code/../Data/

## Base Rule
1. Base rule: 

    if the number of consecutive motions == target > 4:
    
    if the next set of motions < 4 and the motion after == target:
    
    relabel the data


In [4]:
def base_rule(data):
    """ Finds the number of points to relabel based on the base rule.
    
    Uses a consecutive motion threshold of 4. 
    
    Args: 
        data - the virtual game data
        
    Returns:
        total_changed_sum - the total number of data points relabeled
        percentage_changed - the percentage of data points relabeled
    """
    class_data = data[['class', 'targetClass', 'emgChan1', 'emgChan2', 'emgChan3', 'emgChan4', 'emgChan5', 'emgChan6', 'emgChan7', 'emgChan8']].to_numpy()

    # separate into lists based on target class
    values = []
    i_flag = 0
    for i in range(1, len(class_data)):
        if class_data[i-1, 1] == class_data[i, 1]:
            pass
        else:
            values.append(class_data[i_flag:i, :])
            i_flag = i

        if i == len(class_data)-1:
            values.append(class_data[i_flag:i+1, :])

    # find RMS of MAVs at each time point
    j = 0
    for value in values:
        rms = np.zeros((len(value), 1))
        i = 0
        for row in value:
            rms[i] = np.sqrt(np.mean((row[2:])**2))
            i += 1
        value = np.hstack((value, rms))
        values[j] = value
        j += 1

    # find number of points to relabel from base rule
    total_changed = []
    for target in values:    
        # separate target by predicted classes
        expanded_vals = []
        i_flag = 0
        for i in range(1, len(target)):
            if target[i-1, 0] == target[i, 0]:
                pass
            else:
                expanded_vals.append(target[i_flag:i, :])
                i_flag = i

            if i == len(target)-1:
                expanded_vals.append(target[i_flag:i+1, :])

        # get the counts of consecutive motions of predicted classes vs target
        expanded_vals_copy = deepcopy(expanded_vals)
        j = 0
        points = []
        changed = 0
        for part in expanded_vals:
            # check if predicted == target
            if part[0,0] == part[0,1]:
                target_length = len(part)
                # boundary check to make sure not exceeding array size
                if j < len(expanded_vals)-2:
                    next_length = len(expanded_vals[j+1])
                    nextnext_length = len(expanded_vals[j+2])
                    # check if motions after == target
                    if expanded_vals[j+2][0,0] == part[0,1]:
                        points.append([target_length, next_length, nextnext_length])
                        # implement < 4 rule
                        if next_length < 4:
                            expanded_vals_copy[j+1][:,0] = part[0,1]
                            changed += len(expanded_vals_copy[j+1][:,0])
            j += 1

        if not points:
            continue
        print(f"Consecutive motions [target, incorect, target]: \n{points}")
        print(f"Number of relabeled points: {changed}")
        total_changed.append(changed)
        
    total_changed_sum = np.sum(total_changed)
    percentage_changed = np.around((np.sum(total_changed)/len(class_data))*100, 2)
    
    return total_changed_sum, percentage_changed

In [6]:
data = InTheZone[1]
total_changed_sum, percentage_changed = base_rule(data)
print(f"Total changed: {total_changed_sum}")
print(f"Percentage changed: {percentage_changed}%")

Consecutive motions [target, incorect, target]: 
[[167, 6, 30], [30, 2, 7], [7, 4, 4], [4, 1, 4], [4, 4, 1], [1, 2, 5], [31, 2, 2], [2, 1, 1], [1, 1, 7], [1, 3, 30], [30, 4, 2], [2, 1, 3]]
Number of relabeled points: 13
Consecutive motions [target, incorect, target]: 
[[68, 2, 12], [12, 4, 6], [30, 1, 7], [7, 4, 1], [1, 11, 1], [1, 1, 2], [2, 1, 2], [17, 2, 1], [1, 1, 2], [2, 1, 1], [1, 1, 4]]
Number of relabeled points: 10
Consecutive motions [target, incorect, target]: 
[[14, 1, 4], [4, 4, 1], [1, 7, 2], [2, 4, 1], [1, 1, 1], [1, 2, 4], [4, 2, 8], [8, 2, 54], [54, 1, 1], [1, 1, 8], [8, 2, 28], [28, 4, 34], [34, 1, 5], [62, 1, 1]]
Number of relabeled points: 14
Consecutive motions [target, incorect, target]: 
[[148, 1, 54], [54, 1, 32], [32, 7, 3], [3, 1, 2], [2, 1, 1], [1, 2, 18], [18, 3, 1], [1, 1, 3], [3, 1, 3], [4, 1, 40], [40, 3, 2]]
Number of relabeled points: 15
Consecutive motions [target, incorect, target]: 
[[13, 1, 48], [48, 1, 21], [48, 1, 1], [1, 1, 33], [33, 2, 19], [19,

## Augmented Rule
2. Augmented rule:<br>
    Include MAV RMS
    
    **For no motion**<br>
    if the number of consecutive motions == target > 4:<br>
    if the next set of motions == 0 and the motion after == target and MAV RMS falls to either a baseline (stored from no motion classifications) or under a threshold:<br>
    don't include the data or label as no motion class
    
    **For motion**<br>
    if the number of consecutive motions == target > 4:<br>
    if the next set of motions < 4 and the motion after == target and MAV RMS is within some range +/- epsilon:<br>
    relabel the data<br>
    if the next set of motions < 4 and motion after == target, but MAV RMS spiked during incorrect motion then converged:<br>
    label that data as the motion class that was predicted or do nothing

In [7]:
def augmented_rule(data):
    """ Finds the number of points to relabel based on the augmented rule.
    
    Uses a consecutive motion threshold of 4 while bringing MAV RMS into 
    account.
    
    Args: 
        data - the virtual game data
        
    Returns:
        total_changed_sum - the total number of data points relabeled
        percentage_changed - the percentage of data points relabeled
    """
    class_data = data[['class', 'targetClass', 'emgChan1', 'emgChan2', 'emgChan3', 'emgChan4', 'emgChan5', 'emgChan6', 'emgChan7', 'emgChan8']].to_numpy()

    # separate into lists based on target class
    values = []
    i_flag = 0
    for i in range(1, len(class_data)):
        if class_data[i-1, 1] == class_data[i, 1]:
            pass
        else:
            values.append(class_data[i_flag:i, :])
            i_flag = i

        if i == len(class_data)-1:
            values.append(class_data[i_flag:i+1, :])

    # find RMS of MAVs at each time point
    j = 0
    for value in values:
        rms = np.zeros((len(value), 1))
        i = 0
        for row in value:
            rms[i] = np.sqrt(np.mean((row[2:])**2))
            i += 1
        value = np.hstack((value, rms))
        values[j] = value
        j += 1

    # find number of points to relabel
    total_changed = []
    for target in values:    
        # separate target by predicted classes
        expanded_vals = []
        i_flag = 0
        for i in range(1, len(target)):
            if target[i-1, 0] == target[i, 0]:
                pass
            else:
                expanded_vals.append(target[i_flag:i, :])
                i_flag = i

            if i == len(target)-1:
                expanded_vals.append(target[i_flag:i+1, :])

        # get the counts of consecutive motions of predicted classes vs target
        expanded_vals_copy = deepcopy(expanded_vals)
        j = 0
        points = []
        changed = 0
        no_motion = 0
        for part in expanded_vals:
            # check if predicted == target
            if part[0,0] == part[0,1]:
                target_length = len(part)
                # boundary check to make sure not exceeding array size
                if j < len(expanded_vals)-2:
                    next_length = len(expanded_vals[j+1])
                    nextnext_length = len(expanded_vals[j+2])
                    # check if motions after == target
                    if expanded_vals[j+2][0,0] == part[0,1]:
                        points.append([target_length, next_length, nextnext_length])
                        # implement rules
                        if next_length < 4:
                            # no motion relabeling
                            if expanded_vals[j+1][0,0] == 0:
                                value_flag = 0
                                # if RMS goes lower by 500 then relabel as no motion otherwise relabel as motion target
                                RMS_curr = np.sqrt(np.mean((expanded_vals[j][:,-1])**2))
                                RMS_next = np.sqrt(np.mean((expanded_vals[j+1][:,-1])**2))
                                RMS_nextnext = np.sqrt(np.mean((expanded_vals[j+2][:,-1])**2))
                                if RMS_next <= ((RMS_curr-500) and (RMS_nextnext-500)):
                                    expanded_vals_copy[j+1][:,1] = 0
                                    no_motion += len(expanded_vals_copy[j+1][:,1])
                                else:
                                    expanded_vals_copy[j+1][:,0] = part[0,1]
                                    changed += len(expanded_vals_copy[j+1][:,0])
                            # motion relabeling
                            else:
                                expanded_vals_copy[j+1][:,0] = part[0,1]
                                changed += len(expanded_vals_copy[j+1][:,0])
            j += 1

        if not points:
            continue
        print(f"Consecutive motions [target, incorect, target]: \n{points}")
        print(f"Number of relabeled points: {changed+no_motion}")
        print(f"Number of no motion relabels: {no_motion}")
        total_changed.append(changed+no_motion)
        
    total_changed_sum = np.sum(total_changed)
    percentage_changed = np.around((np.sum(total_changed)/len(class_data))*100, 2)
    
    return total_changed_sum, percentage_changed

In [8]:
data = InTheZone[1]
total_changed_sum, percentage_changed = augmented_rule(data)
print(f"Total changed: {total_changed_sum}")
print(f"Percentage changed: {percentage_changed}%")

Consecutive motions [target, incorect, target]: 
[[167, 6, 30], [30, 2, 7], [7, 4, 4], [4, 1, 4], [4, 4, 1], [1, 2, 5], [31, 2, 2], [2, 1, 1], [1, 1, 7], [1, 3, 30], [30, 4, 2], [2, 1, 3]]
Number of relabeled points: 13
Number of no motion relabels: 0
Consecutive motions [target, incorect, target]: 
[[68, 2, 12], [12, 4, 6], [30, 1, 7], [7, 4, 1], [1, 11, 1], [1, 1, 2], [2, 1, 2], [17, 2, 1], [1, 1, 2], [2, 1, 1], [1, 1, 4]]
Number of relabeled points: 10
Number of no motion relabels: 0
Consecutive motions [target, incorect, target]: 
[[14, 1, 4], [4, 4, 1], [1, 7, 2], [2, 4, 1], [1, 1, 1], [1, 2, 4], [4, 2, 8], [8, 2, 54], [54, 1, 1], [1, 1, 8], [8, 2, 28], [28, 4, 34], [34, 1, 5], [62, 1, 1]]
Number of relabeled points: 14
Number of no motion relabels: 0
Consecutive motions [target, incorect, target]: 
[[148, 1, 54], [54, 1, 32], [32, 7, 3], [3, 1, 2], [2, 1, 1], [1, 2, 18], [18, 3, 1], [1, 1, 3], [3, 1, 3], [4, 1, 40], [40, 3, 2]]
Number of relabeled points: 15
Number of no motion r